In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from textblob import TextBlob
from textblob import Word

ModuleNotFoundError: No module named 'textblob'

In [2]:
df = pd.read_csv('spamEmails.csv')
df['EmailText'].head()
df['EmailText'] = df['EmailText'].apply(lambda x:' '.join(x.lower() for x in x.split()))

In [3]:
import nltk
nltk.download("stopwords")

stop_words = stopwords.words('english')
st = PorterStemmer()

df['EmailText'] = df['EmailText'].apply(lambda x : ' '.join(st.stem(word) for word in x.split()))
df['EmailText'].head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tmfaj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


0    go until jurong point, crazy.. avail onli in b...
1                          ok lar... joke wif u oni...
2    free entri in 2 a wkli comp to win fa cup fina...
3    u dun say so earli hor... u c alreadi then say...
4    nah i don't think he goe to usf, he live aroun...
Name: EmailText, dtype: object

In [4]:
import nltk
nltk.download('wordnet')

df['EmailText'] = df['EmailText'].apply(lambda x: ' '.join(Word(word).lemmatize() for word in x.split()))
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tmfaj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


NameError: name 'Word' is not defined

In [ ]:
import sklearn.feature_extraction.text as text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['EmailText'], df['Label'])
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['EmailText'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

In [ ]:
def train_model(classifier, feature_vector_train, label):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    return classifier

In [ ]:
nb_model = train_model(naive_bayes.MultinomialNB(alpha=0.2), xtrain_tfidf, train_y)
predictions = nb_model.predict(xvalid_tfidf)
nb_accuracy = metrics.accuracy_score(predictions, valid_y)
print("Accuracy: ", nb_accuracy)

In [ ]:
from sklearn import model_selection,svm
from sklearn.model_selection import GridSearchCV

cv = CountVectorizer()
features = cv.fit_transform(train_x)

In [ ]:
tuned_parameters = {'kernel': ['linear','rbf'],'gamma': [1e-3, 1e-4], 'C':[1,10,100,1000]}
model = GridSearchCV(svm.SVC(),tuned_parameters)

model.fit(features, train_y)
print(model.best_params_)

features_test = cv.transform(valid_x)
print("Accuracy of the Model is : ", model.score(features_test, valid_y))